In [ ]:
!pip install transformers sentence-transformers
!pip install umap
!pip install umap-learn
!pip install plotly

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Carico il modello pre-addestrato Sentence-BERT
model = SentenceTransformer('all-MiniLM-L6-v2')  # una versione abbastanza leggera

# Supponiamo di avere un elenco di testi (in questo caso poesie ed estratti)
books = [
    "autore Petrarca: Solo e pensoso i più deserti campi  vo mesurando a passi tardi e lenti,  e gli occhi porto per fuggire intenti  ove vestigio uman l'arena stampi.",
    "autore Dante: Nel mezzo del cammin di nostra vita  mi ritrovai per una selva oscura,  ché la diritta via era smarrita.",
    "autore Leopardi: Sempre caro mi fu quest’ermo colle,  e questa siepe, che da tanta parte  dell'ultimo orizzonte il guardo esclude.",
    "autore Foscolo: A egregie cose il forte animo accendono  l'urne de' forti o Pindemonte.",
    "autore Pascoli: San Lorenzo, io lo so perché tanto di stelle  per l'aria tranquilla arde e cade, perché sì gran pianto  nel concavo cielo sfavilla.",
    "autore Montale: Spesso il male di vivere ho incontrato:  era il rivo strozzato che gorgoglia.",
    "autore Ungaretti: Si sta come d’autunno  sugli alberi le foglie.",
    "autore Quasimodo: Ed è subito sera.",
    "autore Carducci: Davanti a San Guido  i cipressi che a Bolgheri alti e schietti  van da San Guido in duplice filar.",
    "autore Gozzano: Non amo che le rose  che non colsi.",
    "autore Saba: La casa di mia madre a Trieste era quieta e scura.",
    "autore D’Annunzio: Settembre, andiamo. È tempo di migrare.",
    "autore Giusti: Ride la gazza, nera sugli aranci,  stormi d’uccelli neri a volo basso.",
    "autore Govoni: I tuoi occhi sono pozzi di fuoco, dove s'affondano i miei sogni.",
    "autore Marinetti: Parole in libertà  illuminano il buio della notte.",
    "autore Boiardo: Amor, che nel mio pensier vive e regna  e 'l suo seggio maggior nel cor mantiene.",
    "autore Guinizelli: Al cor gentil rempaira sempre amore,  com'a l'ausello in selva la verdura.",
    "autore Ariosto: Le donne, i cavallier, l'arme, gli amori,  le cortesie, l'audaci imprese io canto.",
    "autore Tasso: Oh bella libertà, dolce è il tuo raggio  a chi è cresciuto in sen d’una speranza!",
    "autore Parini: Tu, ch’eri sì, nel voler degli altri, e il tuo,  sì pura e alteramente quieta.",
    "autore De Amicis: Scendeva lieve la pioggia d’oro  e i bimbi la correvano a inseguire.",
    "autore Prati: Fino a quel giorno in cui ritornerò  da te, là dove mai ti lasciai.",
    "autore Tommasèo: Che giova all'uom la forza de l'argento?  La povertà è più che il ricco vanto.",
    "autore Pellico: Alla patria io respiro e chiudo in cuore  tutta la dolce libertà del cielo.",
    "autore Manzoni: Ei fu. Siccome immobile,  dato il mortal sospiro.",
    "autore Verga: Una perla nel mar, chi mai la trova  non sa che sarà poi per sempre sua.",
    "autore Caproni: Lo sai che io so che tu sai,  ma ognuno sa quel che sa e tace.",
    "autore Pavese: Verrà la morte e avrà i tuoi occhi.",
    "autore Sereni: Se non faremo i conti,  il vento solleverà i calcinacci.",
    "autore Roversi: Dove volano i passeri all’imbrunire?  L’aria è greve di silenzio.",
    "autore Campana: Cielo stellato sopra di noi,  voce lontana d'un canto triste."
]


# Calcolo l'embedding per ogni libro
book_embeddings = model.encode(books, convert_to_tensor=True)


print("Embedding dei libri:\n", book_embeddings)


Embedding dei libri:
 tensor([[ 0.0465,  0.0983, -0.0444,  ...,  0.0390, -0.0447, -0.0224],
        [-0.0902,  0.0775, -0.0642,  ...,  0.0604, -0.0009, -0.0425],
        [-0.0679,  0.0892,  0.0281,  ...,  0.0073, -0.0027, -0.0199],
        ...,
        [-0.0180,  0.0453, -0.0062,  ...,  0.0718, -0.0204, -0.0808],
        [ 0.0033,  0.0595, -0.0342,  ...,  0.0633, -0.0342, -0.0298],
        [-0.0756,  0.0803, -0.0863,  ...,  0.0222, -0.0155,  0.0504]],
       device='mps:0')


In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

# Riduciozne della dimensionalità degli embedding con t-SNE
tsne = TSNE(n_components=2, perplexity=10)
embeddings_2d = tsne.fit_transform(book_embeddings.cpu().numpy())

df = pd.DataFrame(embeddings_2d, columns=['x', 'y'])
df['text'] = books  # Aggiungi il testo dei libri al DataFrame
df['hover_text'] = df['text'].str.split(':').str[1].str.strip() # Estrai solo la parte dopo il ":"

#  grafico interattivo con Plotly
fig = px.scatter(df, x='x', y='y', hover_data={'text':False, 'hover_text': True}, custom_data=['text'])

fig.update_traces(
    hovertemplate="<br>".join([
        "Testo: %{customdata[0]}",
    ])
)


fig.show()

In [ ]:
# Query dell'utente
query = "vorrei leggere una poesia triste e malinconica"

# Calcolo l'embedding della query
query_embedding = model.encode(query, convert_to_tensor=True)

# Calcolo la similarità coseno tra la query e ogni libro
cosine_scores = util.cos_sim(query_embedding, book_embeddings)

# Ordino in base alla similarità
results = list(enumerate(cosine_scores[0].cpu().numpy()))
results = sorted(results, key=lambda x: x[1], reverse=True)

# Show results 
print("\nRisultati della query:")
for idx, score in results:
    print(f"Libro: {books[idx]} | Similarità: {score:.4f}")



Risultati della query:
Libro: autore Campana: Cielo stellato sopra di noi,  voce lontana d'un canto triste. | Similarità: 0.5994
Libro: autore Petrarca: Solo e pensoso i più deserti campi  vo mesurando a passi tardi e lenti,  e gli occhi porto per fuggire intenti  ove vestigio uman l'arena stampi. | Similarità: 0.5644
Libro: autore Leopardi: Sempre caro mi fu quest’ermo colle,  e questa siepe, che da tanta parte  dell'ultimo orizzonte il guardo esclude. | Similarità: 0.5607
Libro: autore Pellico: Alla patria io respiro e chiudo in cuore  tutta la dolce libertà del cielo. | Similarità: 0.5602
Libro: autore De Amicis: Scendeva lieve la pioggia d’oro  e i bimbi la correvano a inseguire. | Similarità: 0.5544
Libro: autore Tommasèo: Che giova all'uom la forza de l'argento?  La povertà è più che il ricco vanto. | Similarità: 0.5485
Libro: autore Tasso: Oh bella libertà, dolce è il tuo raggio  a chi è cresciuto in sen d’una speranza! | Similarità: 0.5447
Libro: autore Roversi: Dove volano i 